In [4]:
%pip install tensorflow.python

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow.python (from versions: none)
ERROR: No matching distribution found for tensorflow.python

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\carlo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Parámetros del sistema amortiguado
m = 1.0   # Masa
c = 0.5   # Amortiguamiento
k = 2.0   # Rigidez

# Datos de entrenamiento (Tiempo)
t = np.linspace(0, 10, 100).reshape(-1, 1)  # 100 puntos en [0,10]

# Red neuronal para aproximar x(t)
def build_pinn():
    model = keras.Sequential([
        layers.Dense(20, activation='tanh', input_shape=(1,)),
        layers.Dense(20, activation='tanh'),
        layers.Dense(20, activation='tanh'),
        layers.Dense(1)  # Salida x(t)
    ])
    return model

model = build_pinn()

# Definir la función de pérdida física
@tf.function
def pinn_loss(model, t):
    with tf.GradientTape(persistent=True) as tape2:
        with tf.GradientTape(persistent=True) as tape:
            x = model(t)  # Predicción de x(t)
        dx_dt = tape.gradient(x, t)  # Primera derivada
    d2x_dt2 = tape2.gradient(dx_dt, t)  # Segunda derivada
    del tape, tape2
    
    # Ecuación diferencial m*x'' + c*x' + k*x = 0 (sin fuerza externa)
    physics = m * d2x_dt2 + c * dx_dt + k * x
    mse_physics = tf.reduce_mean(tf.square(physics))
    return mse_physics

# Optimizador y entrenamiento
def train(model, t, epochs=5000, lr=0.001):
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss = pinn_loss(model, tf.convert_to_tensor(t, dtype=tf.float32))
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy():.6f}")

# Entrenamos la red
train(model, t)

# Predicción
t_test = np.linspace(0, 10, 100).reshape(-1, 1)
x_pred = model.predict(t_test)


ModuleNotFoundError: No module named 'tensorflow.python'